In [44]:
import torch
from torch import nn
import pytorch_lightning as pl
import numpy as np
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import random
import matplotlib.pyplot as plt

def set_random_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)

In [45]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [46]:
from cyclic_group import CyclicGroup
from nonrelativistic_oscillator import NonRelativisticOscillator
from equivariant_linear_layer import EquivariantLinearLayer
from dataset import NormalDataset
from pipeline import Pipeline

In [47]:
def print_matrix(m):
    for x in m:
        for y in x:
            print(f'{y:+.5f}',end=' ')
        print()

In [48]:
class LinearModel(nn.Module):
    def __init__(self, number_of_nodes, device):
        super().__init__()
        self.n = number_of_nodes
        self.ELL = EquivariantLinearLayer(self.n, device)
            
    def forward(self, z):
        return self.ELL(z)

In [49]:
NUMBER_OF_NODES = 62
NUMBER_OF_SAMPLES = 2**20
BATCH_SIZE = 2**10
TAU = 0.1
train_loader = DataLoader(NormalDataset(number_of_nodes=NUMBER_OF_NODES,
                                        number_of_samples=NUMBER_OF_SAMPLES), 
                          batch_size=BATCH_SIZE, 
                          shuffle=True,
                          num_workers=3)

In [50]:
trainer = pl.Trainer(
    max_epochs=20,
    logger=TensorBoardLogger(save_dir=f"logs/ELM"),
    num_sanity_val_steps=0,
    log_every_n_steps=1,
    accumulate_grad_batches=8
)

LM = LinearModel(number_of_nodes=NUMBER_OF_NODES, device=device)
NR = NonRelativisticOscillator(tau=TAU,
                               number_of_nodes=NUMBER_OF_NODES,
                               device=device)

pipeline=Pipeline(model=LM,
                  criterion=NR.Loss,
                  optimizer_class=torch.optim.Adam,
                  optimizer_kwargs={"lr": 0.01,"weight_decay":0.0})

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [51]:
torch.exp(LM.ELL.weight)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0', grad_fn=<ExpBackward0>)

In [52]:
trainer.fit(model=pipeline, train_dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params | Mode 
----------------------------------------------
0 | model | LinearModel | 32     | train
----------------------------------------------
32        Trainable params
0         Non-trainable params
32        Total params
0.000     Total estimated model params size (MB)


Epoch 19: 100%|██████████| 1024/1024 [00:04<00:00, 219.36it/s, v_num=8, train_loss=105.0]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 1024/1024 [00:04<00:00, 219.02it/s, v_num=8, train_loss=105.0]


In [34]:
mat_model = torch.exp(LM.ELL.weight.to(device).matmul(LM.ELL.matrix.to(device)))
mat_analyt = NR.Eigenvalues()**(-0.5)
print_matrix([mat_model])
print_matrix([mat_analyt])

+3.16085 +0.15790 +2.22182 +2.22182 +1.39566 +1.39566 +0.99342 +0.99342 +0.75834 +0.75834 +0.61911 +0.61911 +0.52147 +0.52147 +0.44966 +0.44966 +0.39679 +0.39679 +0.35678 +0.35678 +0.32393 +0.32393 +0.29766 +0.29766 +0.27623 +0.27623 +0.25756 +0.25756 +0.24198 +0.24198 +0.22874 +0.22874 +0.21783 +0.21783 +0.20797 +0.20797 +0.19956 +0.19956 +0.19211 +0.19211 +0.18605 +0.18605 +0.18076 +0.18076 +0.17568 +0.17568 +0.17185 +0.17185 +0.16836 +0.16836 +0.16537 +0.16537 +0.16319 +0.16319 +0.16123 +0.16123 +0.15961 +0.15961 +0.15862 +0.15862 +0.15814 +0.15814 
+3.16228 +0.15792 +2.22160 +2.22160 +1.40110 +1.40110 +0.99150 +0.99150 +0.76231 +0.76231 +0.61862 +0.61862 +0.52095 +0.52095 +0.45061 +0.45061 +0.39776 +0.39776 +0.35674 +0.35674 +0.32409 +0.32409 +0.29759 +0.29759 +0.27572 +0.27572 +0.25745 +0.25745 +0.24203 +0.24203 +0.22889 +0.22889 +0.21763 +0.21763 +0.20793 +0.20793 +0.19955 +0.19955 +0.19229 +0.19229 +0.18599 +0.18599 +0.18054 +0.18054 +0.17584 +0.17584 +0.17180 +0.17180 +0.16837 

In [35]:
print_matrix([mat_model - mat_analyt])

-0.00143 -0.00001 +0.00022 +0.00022 -0.00544 -0.00544 +0.00191 +0.00191 -0.00396 -0.00396 +0.00049 +0.00049 +0.00051 +0.00051 -0.00095 -0.00095 -0.00096 -0.00096 +0.00004 +0.00004 -0.00016 -0.00016 +0.00007 +0.00007 +0.00051 +0.00051 +0.00010 +0.00010 -0.00004 -0.00004 -0.00015 -0.00015 +0.00019 +0.00019 +0.00004 +0.00004 +0.00001 +0.00001 -0.00018 -0.00018 +0.00006 +0.00006 +0.00022 +0.00022 -0.00016 -0.00016 +0.00005 +0.00005 -0.00001 -0.00001 -0.00011 -0.00011 +0.00008 +0.00008 +0.00002 +0.00002 -0.00015 -0.00015 -0.00011 -0.00011 +0.00002 +0.00002 


In [36]:
generator = torch.distributions.Normal(
            loc=torch.zeros(NUMBER_OF_NODES), 
            scale=torch.ones(NUMBER_OF_NODES))
z = generator.sample((2**20,)).to(device)

In [37]:
x_model, log_jacobian = LM.to(device)(z.to(device))
x_analytic = NR.AnalyticTransformation(z.to(device))
log_jacobian_analytic = NR.AnalyticalLogJacobian()

In [38]:
print('model x^2    =', f'{(x_model**2).mean().item():.3f}')
print('analytic x^2 =', f'{(x_analytic**2).mean().item():.3f}')

model x^2    = 0.501
analytic x^2 = 0.501


In [39]:
print('model log(jacobian)    =', f'{log_jacobian.item():.3f}')
print('analytic log(jacobian) =', f'{log_jacobian_analytic.item():.3f}')

model log(jacobian)    = -74.495
analytic log(jacobian) = -74.477


In [40]:
print('model min loss    =', f'{NR.Loss(x_model, log_jacobian).item():.3f}')
print('analytic min loss =', f'{NR.MinLoss().item():.3f}')

model min loss    = 105.469
analytic min loss = 105.477


In [41]:
NR.expectation_energy(x_model)

tensor([0.2546, 0.2503, 0.5048], device='cuda:0', grad_fn=<CatBackward0>)

In [42]:
NR.expectation_energy(x_analytic)

tensor([0.2519, 0.2506, 0.5025], device='cuda:0')